# Virtualization and OpenShift Workflow Agent

## Overview

This notebook demonstrates how to build an AI-powered multi-agent workflow that interacts with **Virtual Machines (VMs)** and **OpenShift**. The agents process tasks such as listing VMs, retrieving VM details, and managing OpenShift migration plans using custom tools. 

By leveraging a machine learning model, the agents are able to process natural language instructions and execute specific tasks. The workflow is designed to be flexible, allowing different agents to collaborate through a shared state, with task data being persisted using **SQLite**.

## 1. Setup

Before we begin, let's make sure your environment is set up correctly. We'll start by installing the necessary Python packages.

### Installing Required Packages

To get started, you'll need to install a few Python libraries. Run the following command to install them:

In [1]:
%pip install langchain langgraph langgraph-checkpoint-sqlite langchain_community requests termcolor wikipedia arxiv

Note: you may need to restart the kernel to use updated packages.


## 2. Configuring a Simple Model

In this section, we configure the machine learning model that we will use to process tasks. The `ModelService` class manages the interaction with the model (in this case, "llama3.1:8b-instruct-fp16").

### Model Configuration

We initialize the `ModelService` with a specific model configuration, including parameters such as model endpoint, temperature (for controlling randomness), and others. This step enables us to perform model-based tasks using the provided configuration.

In [2]:
from services.model_service import ModelService

# Initialize the service with the model configuration
ollama_service = ModelService(model="llama3.1:8b-instruct-fp16")

## 3. Defining the Shared State

In any multi-agent system, it’s crucial that agents can share information and coordinate their tasks. To achieve this, we use a **shared state** — a centralized data structure that agents read from and update as they perform their respective tasks. This shared state acts as a "memory" that persists between agent actions, enabling collaboration across different stages of the workflow.

In [3]:
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict, Any


class MultiAgentGraphState(TypedDict):
    input: str
    virt_engineer_response: Annotated[list, add_messages]
    ocp_engineer_response: Annotated[list, add_messages]

## 4. Setting Up Tools for the Workflow

In this section, we prepare the tools that will be used by the different agents in our workflow. Each agent has a specific set of tools they use to perform their tasks. Tools can represent any reusable function or utility that an agent can use to accomplish its assigned work.

We are introducing two sets of tools:

1. **OpenShift Tools**: Tools related to OpenShift operations, such as creating or starting a migration plan.
2. **vSphere Tools**: Tools for virtual machine operations, such as listing VMs or retrieving VM details.

### OpenShift Tools

The first set of tools we define is for OpenShift. These tools are used by the OpenShift engineer agent in the workflow. We utilize utility functions to fetch the tools' names and their descriptions, which will later be used in the workflow.

- **`create_migration_plan_tool`**: This tool is responsible for creating a migration plan in OpenShift.
- **`start_migration_plan_tool`**: This tool starts the migration plan that was created.

We then use helper functions (`get_tools_name` and `get_tools_description_json`) to get the names and descriptions of the tools, which will be helpful for dynamically assigning tasks to agents later on.


In [4]:
import json
from utils.general.tools import (
    get_tools_name,
    get_tools_description_json,
)

from tools.openshift.migration_plan import (
    create_migration_plan_tool,
    start_migration_plan_tool,
)

ocp_tools = [create_migration_plan_tool, start_migration_plan_tool]

ocp_tools_name = get_tools_name(ocp_tools)
ocp_tools_description_json = get_tools_description_json(ocp_tools)

ocp_tools_description = json.dumps(ocp_tools_description_json, indent=4)

print(ocp_tools_description)

OpenShift Service connected. API is healthy.
[
    {
        "name": "create_migration_plan_tool",
        "description": "A tool to create a migration plan for moving virtual machines (VMs) using the forklift.konveyor.io API. This tool only requires VM names, and the corresponding IDs are fetched automatically.",
        "parameters": {
            "vm_names": {
                "param_type": "str",
                "description": "A list of VM names to migrate. This arg is mandatory.",
                "required": true
            },
            "name": {
                "param_type": "str",
                "description": "The name of the migration plan. This arg is mandatory.",
                "required": true
            },
            "source": {
                "param_type": "str",
                "description": "The source provider. Default is 'vmware'.",
                "required": false
            }
        }
    },
    {
        "name": "start_migration_plan_tool",
        "des

/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.rosa-tttcp.1fe1.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### vSphere Tools

Similarly, we define a set of tools for managing virtual machines. These tools will be used by the virtual engineer agent in the workflow.

- **`list_vms`**: This tool lists all the virtual machines available in the vSphere environment.
- **`retrieve_vm_details`**: This tool retrieves details about a specific virtual machine.

Like the OpenShift tools, we extract the names and descriptions of these tools for later use in the workflow.

In [5]:
from tools.vsphere.vms import list_vms, retrieve_vm_details

virt_tools = [list_vms, retrieve_vm_details]

virt_tools_name = get_tools_name(virt_tools)
virt_tools_description_json = get_tools_description_json(virt_tools)

virt_tools_description = json.dumps(virt_tools_description_json, indent=4)

print(virt_tools_description)

[
    {
        "name": "list_vms",
        "description": "A wrapper around a vSphere utility for listing all available virtual machines (VMs). Useful for retrieving a list of VMs from the connected vSphere environment.",
        "parameters": {}
    },
    {
        "name": "retrieve_vm_details",
        "description": "A wrapper around a vSphere utility for extracting detailed information about a specific virtual machine (VM). Useful for retrieving the VM's operating system, resource allocations, and network configurations based on the provided VM name.",
        "parameters": {
            "vm_name": {
                "param_type": "str",
                "description": "The name of the virtual machine to retrieve details for.",
                "required": true
            }
        }
    }
]


## 5. Configuring Agent Nodes

Next, we will configure the virtual engineer and OpenShift engineer agents. These agents will be responsible for performing specific tasks in the workflow by utilizing the tools we just defined.

For each agent, we define:
- A **role** (e.g., `virt_engineer` or `ocp_engineer`).
- A set of **tools** that the agent can use to complete its tasks.
  
The agents will respond to the tasks dynamically, leveraging the tools to produce results.

In [6]:
from agent.react_agent import ReactAgent
from utils.general.helpers import remove_prefix


# Process a list of tasks for the Virtual Engineer.
def virt_eng_node_function(state: MultiAgentGraphState, tasks: list[dict]):
    virt_agent = ReactAgent(
        state=state,
        role="virt_engineer",
        tools=virt_tools,
        ollama_service=ollama_service,
    )

    # Process each task and collect the responses
    responses = []
    for task in tasks:
        task_id = task["task_id"]
        task_description = task["task_description"]
        response = virt_agent.react(user_request=task_description)
        processed_response = remove_prefix(response["virt_engineer_response"].content)

        # Append the task_id and its corresponding response
        responses.append(str({"task_id": task_id, "response": processed_response}))

    return {"virt_engineer_response": responses}


# Process a list of tasks for the OpenShift Engineer.
def ocp_eng_node_function(state: MultiAgentGraphState, tasks: list[dict]):
    ocp_agent = ReactAgent(
        state=state,
        role="ocp_engineer",
        tools=ocp_tools,
        ollama_service=ollama_service,
    )

    # Process each task and collect the responses
    responses = []
    for task in tasks:
        task_id = task["task_id"]
        task_description = task["task_description"]
        response = ocp_agent.react(user_request=task_description)
        processed_response = remove_prefix(response["ocp_engineer_response"].content)

        # Append the task_id and its corresponding response
        responses.append(str({"task_id": task_id, "response": processed_response}))

    return {"ocp_engineer_response": responses}

## 6. Workflow

In this section, we will finalize the workflow, integrate the agents, and run the graph. The workflow involves setting up virtual machine-related tasks for the **Virtual Engineer** and OpenShift-related tasks for the **OpenShift Engineer**. Both agents will receive lists of tasks and perform their respective operations using the tools provided. We'll use the `create_graph` function to set up the agent nodes, and `run_workflow` to execute the workflow.

In [7]:
from state.state_graph import create_graph, run_workflow
from memory.sqlite_saver import initialize_memory

vm_name = "database-tttcp"
migration_plan_name = "migration-plan"

virt_tasks = [
    {
        "task_id": "virt_task_1",
        "task_description": "This is a SINGLE task: List all VMs! Just do this and nothing else! DO NOT RETRIEVE THE DETAILS!",
    },
    {
        "task_id": "virt_task_2",
        "task_description": f"This is a SINGLE task: List the details for the vm named: {vm_name}! Just do this and nothing else! DO NOT RETRIEVE THE DETAILS!",
    },
]

ocp_tasks = [
    {
        "task_id": "ocp_task_1",
        "task_description": f"You should execute only one task. THIS IS A SINGLE TASK: Use the create_migration_plan_tool to create a migration plan. Details: plan_name: {migration_plan_name}, vm_name:'{vm_name}'! REMEMBER: DON'T DO ANY ADDITIONAL STEP! DO NOT START OR EXECUTE THE MIGRATION YET!",
    },
    {
        "task_id": "ocp_task_2",
        "task_description": f"You should execute only one task. THIS IS A SINGLE TASK: Use the start_migration_plan_tool to start the {migration_plan_name}! Details: plan_name: {migration_plan_name}. REMEMBER: DON'T DO ANY ADDITIONAL STEP! DO NOT CREATE THE MIGRATION!",
    },
]

# Define a dictionary to map node names to their corresponding functions
agent_nodes = {
    "virt_engineer_node": lambda state: virt_eng_node_function(
        state=state, tasks=virt_tasks
    ),
    "ocp_engineer_node": lambda state: ocp_eng_node_function(
        state=state, tasks=ocp_tasks
    ),
}

memory = initialize_memory(":memory:")

# Step 2: Create the graph
graph = create_graph(
    MultiAgentGraphState,
    agent_nodes,
)

# Step 3: Define the input query and config
query = "Who's the USA's President?"
config = {"configurable": {"thread_id": "1"}}

# Step 4: Run the workflow
run_workflow(
    graph=graph,
    input_query=query,
    config=config,
    memory=memory,
    iterations=10,
    verbose=True,
)

<|start_header_id|>user<|end_header_id|>

This is a SINGLE task: List all VMs! Just do this and nothing else! DO NOT RETRIEVE THE DETAILS!<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

{
    "thought": "To list all virtual machines, I should use the list_vms tool to retrieve the list of available VMs.",
    "action": "list_vms",
    "action_input": {}
}<|eot_id|>
<|python_tag|>list_vms.call({})
<|eom_id|>
Connected successfully to vCenter at vcsrs00-vc.infra.demo.redhat.com
VMs in the vSphere environment: roadshow-tpl-winweb01, winweb02-tttcp, winweb01-tttcp, database-tttcp, haproxy-tttcp, rhel93-tpl, roadshow-tpl-database, roadshow-tpl-winweb02, rhel86-tpl, rhel9-tpl
<|start_header_id|>ipython<|end_header_id|>

['roadshow-tpl-winweb01', 'winweb02-tttcp', 'winweb01-tttcp', 'database-tttcp', 'haproxy-tttcp', 'rhel93-tpl', 'roadshow-tpl-database', 'roadshow-tpl-winweb02', 'rhel86-tpl', 'rhel9-tpl']<|eot_id|>
<|start_header_id|>ipython<|end_header_id|>

{
    "observation": [
 

/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.rosa-tttcp.1fe1.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


OpenShift Service connected. API is healthy.

1/8 Connected to Openshift...



/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forklift-inventory-openshift-mtv.apps.rosa-tttcp.1fe1.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



2/8 Now I have the source provider ID f19517d3-9207-4392-be1c-910b0724ebec...



...


7/8 Now I have the Storage and Network maps [{'id': 'vm-98459', 'name': 'database-tttcp'}]...



/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forklift-inventory-openshift-mtv.apps.rosa-tttcp.1fe1.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forklift-inventory-openshift-mtv.apps.rosa-tttcp.1fe1.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ap


8/8 Migration plan RESPONSE - {'apiVersion': 'forklift.konveyor.io/v1beta1', 'kind': 'Plan', 'metadata': {'annotations': {'populatorLabels': 'True'}, 'creationTimestamp': '2024-09-13T19:21:44Z', 'generation': 1, 'managedFields': [{'apiVersion': 'forklift.konveyor.io/v1beta1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:spec': {'.': {}, 'f:map': {'.': {}, 'f:network': {}, 'f:storage': {}}, 'f:provider': {'.': {}, 'f:destination': {}, 'f:source': {}}, 'f:targetNamespace': {}, 'f:vms': {}}}, 'manager': 'OpenAPI-Generator', 'operation': 'Update', 'time': '2024-09-13T19:21:44Z'}], 'name': 'migration-plan', 'namespace': 'openshift-mtv', 'resourceVersion': '3075108', 'uid': '06e9230d-7650-4aea-b905-a2952382965b'}, 'spec': {'map': {'network': {'apiVersion': 'forklift.konveyor.io/v1beta1', 'kind': 'NetworkMap', 'name': 'vmware-zwk6j', 'namespace': 'openshift-mtv', 'uid': 'a801c689-1584-4871-b495-a3032c11533a'}, 'storage': {'apiVersion': 'forklift.konveyor.io/v1beta1', 'kind': 'StorageMap', 'name

/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.rosa-tttcp.1fe1.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


OpenShift Service connected. API is healthy.

1/4 Connected to Openshift...


2/4 Retrieved migration plan UID: 06e9230d-7650-4aea-b905-a2952382965b...


3/4 Migration plan is ready. Starting migration...


4/4 Migration started: {'apiVersion': 'forklift.konveyor.io/v1beta1', 'kind': 'Migration', 'metadata': {'creationTimestamp': '2024-09-13T19:21:57Z', 'generateName': 'migration-plan-', 'generation': 1, 'managedFields': [{'apiVersion': 'forklift.konveyor.io/v1beta1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:metadata': {'f:generateName': {}, 'f:ownerReferences': {'.': {}, 'k:{"uid":"06e9230d-7650-4aea-b905-a2952382965b"}': {}}}, 'f:spec': {'.': {}, 'f:plan': {}}}, 'manager': 'python-requests', 'operation': 'Update', 'time': '2024-09-13T19:21:57Z'}], 'name': 'migration-plan-fw44w', 'namespace': 'openshift-mtv', 'ownerReferences': [{'apiVersion': 'forklift.konveyor.io/v1beta1', 'kind': 'Plan', 'name': 'migration-plan', 'uid': '06e9230d-7650-4aea-b905-a2952382965b'}], 'resourceVersion': 